# **数据清洗和准备**
---


## **处理缺失值**

> **_None值也属于NaN_**   
> **_当进行数据清洗以进行分析时，最好直接对缺失数据进行分析，以判断数据采集的问题或缺失数据可能导致的偏差。_**

In [ ]:
import numpy as np 
import pandas as pd
string_data = pd.Series(['aardvark', 'artichoke', np.nan, 'avocado'])
string_data.isnull()

> **NaN值处理方式**  
> **dropna | 移除NaN值**   
> **fillna | 填充NaN值**  
> **isnull | 判断null值**   
> **notnull | 过滤null值** 


### **dropna()过滤缺失值**

> **Series的NaN值处理方式**

In [ ]:
from numpy import nan as NA
data = pd.Series([1, NA, 3.5, NA, 7])
data.dropna()

In [ ]:
data[data.notnull()]

> **Dataframe的NaN值处理方式：how='all'只删除全部NAN值的行或者列**

In [ ]:
data = pd.DataFrame([[1., 6.5, 3.], [1., NA, NA],
                    [NA, NA, NA], [NA, 6.5, 3.]])

cleaned = data.dropna()

data

In [ ]:
data.dropna(how='all') 

In [ ]:
data[4] = NA
data.dropna(axis=1,how='all') # 传入axis=1 对列操作

> **保留至少有n个非NaN数据的行/列（用axis=0或1表示）**  
> **简单的理解：这一行除去NA值，剩余数值的数量大于等于n，便显示这一行。**  
> **df.dropna(thresh=2):留下一部分观测数据**

In [ ]:
df = pd.DataFrame(np.random.randn(7, 3))

df.iloc[:4, 1] = NA

df.iloc[:2, 2] = NA

df

In [ ]:
df.dropna(thresh=3)

### **填充数据**

In [ ]:
df.fillna(0)

In [ ]:
df.fillna({1: 0.5, 2: 0})

> **reindexing有效的那些插值方法也可用于fillna**

In [ ]:
df.fillna(method='ffill')
df.fillna(method='ffill', limit=2)

> **可以传入很多聚合函数**

In [ ]:
data = pd.Series([1., NA, 3.5, NA, 7])
data.fillna(data.mean())

> **fillna的参数**  
> **value:填充的值或者字典对象**  
> **method：插值方式，默认ffill**  
> **axis:轴**
> **inplace:在对象上修改，不产生副本**  
> **limit：连续填充的最大数量**

---
## **数据转换**

### **移除重复值**

> **df.duplicated()查找重复值**

In [ ]:
data = pd.DataFrame({'k1': ['one', 'two'] * 3 + ['two'],
                       'k2': [1, 1, 2, 3, 3, 4, 4]})
data.duplicated()

> **df.drop_duplicates()删除重复值**

In [ ]:
data.drop_duplicates()

In [ ]:
data['v1'] = range(7)
data.drop_duplicates(['k1']) # 根据固定列删除重复值
data.drop_duplicates(['k1', 'k2'], keep='last') # 默认保留最后一个重复值

### **利用函数或映射转换数据**

> **S.map()接受函数或者映射关系的字典型对象**

In [ ]:
data = pd.DataFrame({'food': ['bacon', 'pulled pork', 'bacon',
                                'Pastrami', 'corned beef', 'Bacon',
                                'pastrami', 'honey ham', 'nova lox'],
                                'ounces': [4, 3, 12, 6, 7.5, 8, 3, 5, 6]})

meat_to_animal = {
                    'bacon': 'pig',
                    'pulled pork': 'pig',
                    'pastrami': 'cow',
                    'corned beef': 'cow',
                    'honey ham': 'pig',
                    'nova lox': 'salmon'
                    }

lowercased = data['food'].str.lower() # 字符串大写转小写
data['animal'] = lowercased.map(meat_to_animal)
data

> **S.map()一次性搞定**

In [ ]:
# (lambda x,y:x*y) 是匿名函数，常规函数的简写版
data['food'].map(lambda x:meat_to_animal[x.lower()])

### **替换值**

> **replace(x,y)替换值**  
> 笔记：data.replace方法与data.str.replace不同，后者做的是字符串的元素级替换。我们会在后面学习Series的字符串方法。

In [ ]:
data = pd.Series([1., -999., 2., -999., -1000., 3.])

data


In [ ]:
data.replace(-999, np.nan)
data.replace([-999, -1000], np.nan) # 替换多个值
data.replace([-999, -1000], [np.nan,0]) # 替换成不同的值
data.replace({-999: np.nan, -1000: 0}) # 也可以传入字典

### **重命名轴索引**

> **index.map()通过函数更改轴索引名**

In [ ]:
data = pd.DataFrame(np.arange(12).reshape((3, 4)),
                     index=['Ohio', 'Colorado', 'New York'],
                     columns=['one', 'two', 'three', 'four'])
data

In [ ]:
transform = lambda x:x[:4].upper() # 注意这里把index列表的每个值拿出来取[:4]
data.index = data.index.map(transform)
data 

> **df.rename()重命名索引**

In [ ]:
data.rename(index=str.title, columns=str.upper)

> **df.rename()用字典对象更改轴标签**  
> 特别说明一下，rename可以结合字典型对象实现对部分轴标签的更新：

In [ ]:
data.rename(index={'OHIO': 'INDIANA'},
             columns={'three': 'peekaboo'})

> **_df.rename()不更改源数据，复制一份新的_**  
> **_df.reindex()更改源数据，在源数据上修改_**

In [ ]:
data.rename(index={'OHIO': 'INDIANA'},
            columns={'three': 'peekaboo'},inplace=True)
                                                # inplace=True 修改源数据
data

### **离散化cut() + qcut()和面元划分——其实就是区间/数据桶的概念**

> **pd.cut(arg1,arg2)传入指定区间，根据样本指定区间划分**

In [2]:
import pandas as pd
ages = [20, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32]
bins = [18, 25, 35, 60, 100]
cats = pd.cut(ages,bins)
cats

[(18, 25], (18, 25], (18, 25], (25, 35], (18, 25], ..., (25, 35], (60, 100], (35, 60], (35, 60], (25, 35]]
Length: 12
Categories (4, interval[int64, right]): [(18, 25] < (25, 35] < (35, 60] < (60, 100]]

In [3]:
cats.codes # 年龄数据标签

array([0, 0, 0, 1, 0, 0, 2, 1, 3, 2, 2, 1], dtype=int8)

In [4]:
cats.categories

IntervalIndex([(18, 25], (25, 35], (35, 60], (60, 100]], dtype='interval[int64, right]')

In [5]:
pd.value_counts(cats) # 计数值

(18, 25]     5
(25, 35]     3
(35, 60]     3
(60, 100]    1
dtype: int64

> **right=False闭合方式**

In [ ]:
pd.cut(ages, [18, 26, 36, 61, 100], right=False) # 右边闭合

> **pd.cut(ages, bins, labels=group_names)自定义组名**

In [ ]:
group_names = ['Youth', 'YoungAdult', 'MiddleAged', 'Senior']
pd.cut(ages, bins, labels=group_names)

> **pd.cut(data, 4, precision=2)传入固定数值**

In [ ]:
import pandas as pd
import numpy as np
data = np.random.rand(20)
pd.cut(data,4,precision=2) # 保留两位小数

> **pd.qcut(df,arg)根据数据的结果等分划分**

In [ ]:
data = np.random.randn(1000)  # Normally distributed
cats = pd.qcut(data, 4)  # Cut into quartiles
cats

### **检测和过滤异常值**

In [ ]:
import numpy as np 
import pandas as pd
data = pd.DataFrame(np.random.randn(1000,4))
data.describe()

In [ ]:
col = data[2]
col[np.abs(col)>3] # 过滤数值

In [ ]:
data[(np.abs(data) > 3).any(1)] 

In [ ]:
data[np.abs(data) > 3] = np.sign(data) * 3
data.describe()

In [ ]:
np.sign(data).head()

### **排列和随机采样**

In [ ]:
df = pd.DataFrame(np.arange(5 * 4).reshape((5, 4)))
df

> **_np.random.permuration()生成序列_**

In [ ]:
sampler = np.random.permutation(5)
sampler

> **_df.take()使用序列排序_**

In [ ]:
df.take(sampler)

> **df.sample(n=3)不替换的方式选取**

In [ ]:
df.sample(n=3)

In [ ]:
choices = pd.Series([5, 7, -1, 6, 4])
draws = choices.sample(n=10, replace=True)
draws

### **_计算指标/哑变量_**

In [ ]:
df = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'],
                       'data1': range(6)})

pd.get_dummies(df['key'])
dummies = pd.get_dummies(df['key'], prefix='key')
df_with_dummy = df[['data1']].join(dummies)

mnames = ['movie_id', 'title', 'genres']
movies = pd.read_table('datasets/movielens/movies.dat', sep='::',
                        header=None, names=mnames)

movies[:10]


In [ ]:
all_genres = []
for x in movies.genres:
            all_genres.extend(x.split('|'))
genres = pd.unique(all_genres)
genres

---
## **字符串操作**


### **字符串对象方法**

> **val.split()拆分字符串**

In [ ]:
val = 'a,b,  guido'
val.split(',')

> **x.strip()去除空字符和换行符**

In [ ]:
pieces = [x.strip() for x in val.split(',')]
pieces

In [ ]:
first, second, third = pieces
first + '::' +second + '::' +third

In [ ]:
'::'.join(pieces)

In [ ]:
'guido' in val

In [ ]:
val.index(',')

In [ ]:
val.find(':')

In [ ]:
val.count(',')

In [ ]:
val.replace(',','::')

In [ ]:
val.replace(',','')

![字符串方法](img/%E5%AD%97%E7%AC%A6%E4%B8%B2%E6%96%B9%E6%B3%95.png)

### **正则表达式**

### **正则表达式**

In [ ]:
data = {'Dave': 'dave@google.com', 'Steve': 'steve@gmail.com',
        'Rob': 'rob@gmail.com', 'Wes': np.nan}

data = pd.Series(data)
data

In [ ]:
data.isnull()

> **_data.map，所有字符串和正则表达式方法都能被应用于（传入lambda表达式或其他函数）各个值，但是如果存在NA（null）就会报错。_**

In [ ]:
data.str.contains('gmail') # 检查是否包含

In [ ]:
# 也可以使用正则表达式 pip install pattern
import pattern
data.str.findall(pattern, flags=re.IGNORECASE)

In [ ]:
matches = data.str.match(pattern, flags=re.IGNORECASE)
matches

In [ ]:
matches.str.get(1)

In [ ]:
matches.str[0]

In [ ]:
data.str[:5]

![](img/pandas%E5%AD%97%E7%AC%A6%E4%B8%B2%E6%96%B9%E6%B3%95.png)